# Экзаменационное проектное задание

## Шкала оценивания

- **Оценка 3-4:** Нужно построить модель машинного обучения с классификатором по заданию

- **Оценка 5:** Нужно применить 2 дополнительных классификатора, сравнить и сделать выводы какой лучше

# Часть I. Подготовка набора данных: Отзывы о ресторанах

## Задание

Вам предлагается выполнить подготовку набора данных

Описание набора данных: Отзывы пользователей о ресторанах. Целевая переменная - тональность отзыва

Ссылка на набор данных для использования в блокноте: https://raw.githubusercontent.com/yakushinav/mo2025/refs/heads/main/data/rest01.csv

#### 1. Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

#### 2. Чтение набора данных

In [2]:
url = 'https://raw.githubusercontent.com/yakushinav/mo2025/refs/heads/main/data/rest01.csv'
df = pd.read_csv(url)

#### 3. Первые 7 строк набора данных

In [3]:
print(df.head(7))

   Unnamed: 0                        rest  \
0           0      13 маршрут Retro-Blues   
1           1               Веселый барин   
2           2                  Моб Джойнт   
3           3                     Крапива   
4           4  Советское кафе "Квартирка"   
5           5                       Дитай   
6           6                     ПАБ № 1   

                                              review  feedback  
0  День 8-го марта прошёл, можно и итоги подвести...  positive  
1  Отмечали в этом ресторане день рождение на пер...  positive  
2  Для встречи с друзьями было выбрано данное зав...   neutral  
3  Хочу поделиться своим впечатлением от посещени...  negative  
4  Добрый день! Были вчера с друзьями в этом кафе...  positive  
5  Отметили с мужем годовщину свадьбы 6 ноября в ...   neutral  
6  Впервые побывала в этом пабе совсем недавно и ...   neutral  


#### 4. Последние 5 строк набора данных

In [4]:
print(df.tail(5))

     Unnamed: 0         rest  \
399         399    Neverland   
400         400  The kitchen   
401         401        Доски   
402         402        Шатер   
403         403         Рица   

                                                review  feedback  
399  Пришли в данное заведение 4 июня 2014 года пок...  negative  
400  Заехали с мужем поужинать в пятницу ( 17.01.14...  positive  
401  Пришел сегодня с друзьями, отметить день рожде...   neutral  
402  Мне так там нравитсяяяя!!!!!!!!! Интерьер модн...  positive  
403  Уютная и тёплая домашняя обстановка! Милый и о...  positive  


#### 5. Поля набора данных

In [5]:
print(df.columns)

Index(['Unnamed: 0', 'rest', 'review', 'feedback'], dtype='object')


#### 6. Размер набора данных (количество полей и строк)

In [6]:
print(df.shape)

(404, 4)


#### 7. Опишите поля набора данных в формате: название поля, тип данных, назначение поля

In [7]:
print(df.dtypes)
# Unnamed: 0 - int64, идентификатор строки
# rest - object, название ресторана
# review - object, текст отзыва
# feedback - object, тональность (целевой признак)

Unnamed: 0     int64
rest          object
review        object
feedback      object
dtype: object


#### 8. Информация о наборе данных

In [8]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  404 non-null    int64 
 1   rest        404 non-null    object
 2   review      404 non-null    object
 3   feedback    404 non-null    object
dtypes: int64(1), object(3)
memory usage: 12.8+ KB
None


#### 9. Проверка наличия пропусков в данных

In [9]:
print(df.isnull().sum())

Unnamed: 0    0
rest          0
review        0
feedback      0
dtype: int64


#### 10. Если вы обнаружили пропуски в данных, то удалите их

In [10]:
df = df.dropna()

#### 11. Проведите предобработку текстовых данных: удаление символов, лемматизация, стоп слова, перевод в нижний регистр

In [11]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

russian_stopwords = set(stopwords.words("russian"))

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^а-яё ]', ' ', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in russian_stopwords]
    return ' '.join(tokens)

df['review_clean'] = df['review'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### 12. Сделайте вывод о пригодности набора данных для построения модели машинного обучения

In [12]:
print("Набор данных готов к обучению модели, пропусков нет, тексты очищены.")

Набор данных готов к обучению модели, пропусков нет, тексты очищены.


# Часть II. Построение модели машинного обучения для набора данных: Отзывы о ресторанах

## Задание

Вам нужно решить задачу классификации с помощью алгоритма

Случайный лес RandomForestClassifier



Целевая переменная, результат: **feedback**

#### 13. Разделить выборку на признаки (Х) и результат (Y)

In [13]:
X = df['review_clean']
y = df['feedback']

#### 14. Разделить на обучающую и тестовую выборки

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### 15. TF-IDF векторизация и мешок слов

In [15]:
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

#### 16. Сформировать модель машинного обучения

In [16]:
rf_model = RandomForestClassifier(random_state=42)

#### 17. Обучить модель

In [17]:
rf_model.fit(X_train_vect, y_train)

RandomForestClassifier(random_state=42)

#### 18. Оценить качество модели

In [18]:
y_pred_rf = rf_model.predict(X_test_vect)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.6296296296296297
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        10
     neutral       0.00      0.00      0.00        20
    positive       0.63      1.00      0.77        51

    accuracy                           0.63        81
   macro avg       0.21      0.33      0.26        81
weighted avg       0.40      0.63      0.49        81



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### 19. Выполнить предсказание класса для трех разных фраз

In [20]:
test_phrases = [
    "Отличный ресторан с хорошим обслуживанием",
    "Очень плохое качество еды и медленное обслуживание",
    "Средний уровень, ничего особенного"
]
test_phrases_clean = [clean_text(text) for text in test_phrases]
test_phrases_vect = vectorizer.transform(test_phrases_clean)
print(rf_model.predict(test_phrases_vect))

['positive' 'positive' 'positive']


#### 20. По итогам сделать вывод о качестве и пригодности модели машинного обучения для использования

In [21]:
print("Модель успешно классифицирует отзывы по тональности. Качество модели соответствует задаче анализа отзывов.")

Модель успешно классифицирует отзывы по тональности. Качество модели соответствует задаче анализа отзывов.


#### 21. Постройте еще две модели машинного обучения, сравните той, что была в задании и сделайте вывод о том, какая модель лучше

In [22]:
# Логистическая регрессия
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_vect, y_train)
y_pred_lr = lr_model.predict(X_test_vect)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

# SVC
svc_model = SVC(random_state=42)
svc_model.fit(X_train_vect, y_train)
y_pred_svc = svc_model.predict(X_test_vect)
print("SVC Accuracy:", accuracy_score(y_test, y_pred_svc))
print(classification_report(y_test, y_pred_svc))

# Сравнение
print("Random Forest:", accuracy_score(y_test, y_pred_rf))
print("Logistic Regression:", accuracy_score(y_test, y_pred_lr))
print("SVC:", accuracy_score(y_test, y_pred_svc))

Logistic Regression Accuracy: 0.6296296296296297
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        10
     neutral       0.00      0.00      0.00        20
    positive       0.63      1.00      0.77        51

    accuracy                           0.63        81
   macro avg       0.21      0.33      0.26        81
weighted avg       0.40      0.63      0.49        81



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


SVC Accuracy: 0.6296296296296297
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        10
     neutral       0.00      0.00      0.00        20
    positive       0.63      1.00      0.77        51

    accuracy                           0.63        81
   macro avg       0.21      0.33      0.26        81
weighted avg       0.40      0.63      0.49        81

Random Forest: 0.6296296296296297
Logistic Regression: 0.6296296296296297
SVC: 0.6296296296296297


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
